In [4]:
import numpy as np
from sklearn.feature_extraction import image
import imageio
from matplotlib.pyplot import imshow
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import os
import random
from tqdm import tqdm

# use this to split a dataset in one folder into training and validation folders
def splitAndMoveData(data_folder, train_folder="data/training/", val_folder="data/validation/", val_split=0.1):
    filenames = os.listdir(data_folder)
    random.shuffle(filenames)

    train_files = filenames[:len(filenames) - int(len(filenames) * val_split)]
    val_files = filenames[len(filenames) - int(len(filenames) * val_split):]

    print("{} training files and {} validation files".format(len(train_files), len(val_files)))

    for f in tqdm(train_files):
        os.replace(data_folder + f, train_folder + f)

    for f in tqdm(val_files):
        os.replace(data_folder + f, val_folder + f)

# use this to get the training and validation file paths for the data generators
def getFilePaths(train_folder="data/training/", val_folder="data/validation/"):
    train_files = [train_folder + f for f in os.listdir(train_folder)]
    val_files = [val_folder + f for f in os.listdir(val_folder)]

    print("{} training files and {} validation files".format(len(train_files), len(val_files)))

    return train_files, val_files



In [6]:
import os
import keras

train_files, val_files = getFilePaths()

patch_size = (100, 100)
y_stride = 25
x_stride = 25
batch_size = 32 # number of images to get patches from
patches_per_batch = 64 # total number of patches per batch of images
max_patches = patches_per_batch // batch_size   # number of patches to get from each image in the batch
train_steps = len(train_files) // batch_size
val_steps =  len(val_files) // batch_size

32850 training files and 3650 validation files


In [12]:
import cv2
import imageio
import numpy as np
from sklearn.feature_extraction import image
import keras
from PIL import Image
import random

class data_generator(keras.utils.Sequence):
    def __init__(self):
        self.filenames = train_files if self.train else val_files
        self.batch_size = batch_size
        self.patch_size = patch_size
        self.patch_h = patch_size[1]
        self.patch_w = patch_size[0]
        self.max_patches = max_patches
        self.x_stride = x_stride
        self.y_stride = y_stride
        

    def __len__(self) :
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(np.int)
    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        while 1:
            for item in (self[i] for i in range(len(self))):
                yield item

    def load(self, path):
        #img = Image.open(path)
        #img = np.array(img, dtype=np.uint8)
        img = keras.preprocessing.image.load_img(path)
        img = keras.preprocessing.image.img_to_array(img)

        return img / 255.
    
    def pixalate_image(self, image, scale_percent = 40):
      og_w = image.shape[1]
      og_h = image.shape[0]
      width = int(image.shape[1] * scale_percent / 100)
      height = int(image.shape[0] * scale_percent / 100)
      dim = (width, height)

      small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
      
      # scale back to original size
      width = og_w
      height = og_h
      dim = (width, height)

      low_res_image = cv2.resize(small_image, dim, interpolation = cv2.INTER_AREA)

      return low_res_image
    
    def getPatchRanges(self, img):
      ranges = []
      for y in range(self.patch_h, img.shape[0] + 1 + (self.y_stride - (img.shape[0] % self.y_stride)), self.y_stride):
        if y > img.shape[0]:
          y = img.shape[0]

        for x in range(self.patch_w, img.shape[1] + 1 + (self.x_stride - (img.shape[1] % self.x_stride)), self.x_stride):
          if x > img.shape[1]:
            x = img.shape[1]

          ranges.append([y - self.patch_h, y, x - self.patch_w, x])

      return ranges

    def extractPatches(self, img, ranges):
      return [img[y0 : y1, x0 : x1] for y0, y1, x0, x1 in ranges]

    def __getitem__(self, idx):
        batch_ids = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        og_images = [self.load(f) for f in batch_ids]#np.array(list(map(self.load, batch_ids)))
        pixelated_images = [self.pixalate_image(img, 50) for img in og_images]#np.array(list(map(self.pixalate_image, images)))

        if not self.train:
          np.random.seed(0)

        target_patches = []
        input_patches = []
        for i, img in enumerate(og_images):
          if not self.train:
            ranges = self.getPatchRanges(img)
            idxs = np.random.choice(list(range(len(ranges))), size = max_patches)
            ranges = [ranges[i] for i in idxs]
          else:
            ranges = random.sample(self.getPatchRanges(img), max_patches)

          target_patches.extend(self.extractPatches(img, ranges))
          input_patches.extend(self.extractPatches(pixelated_images[i], ranges))
        
        target_patches = np.array(target_patches)
        input_patches = np.array(input_patches)
        return input_patches, target_patches
        
class train_generator(data_generator):
  def __init__(self):
    self.train = True
    super().__init__()

class val_generator(data_generator):
  def __init__(self):
    self.train = False
    super().__init__()

# loader functions for the generators needed by tensorflow
# in order to use interleave   
def get_train_dataset(self):
    train = True
    self = tf.data.Dataset.from_generator(
        train_generator,
        output_types = (tf.float32, tf.float32))
    return self

def get_val_dataset(self):
    train = False
    self = tf.data.Dataset.from_generator(
        val_generator,
        output_types = (tf.float32, tf.float32))
    return self

In [9]:
#@title
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add
from tensorflow.keras.callbacks import EarlyStopping

def getModel():
  Input_img = Input(shape = patch_size + (3,))  
      
  #encoding architecture
  x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)
  x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
  x2 = MaxPool2D( (2, 2))(x2)
  encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

  # decoding architecture
  x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
  x3 = UpSampling2D((2, 2))(x3)
  x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
  x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
  decoded = Conv2D(3, (3, 3), padding='same')(x1)

  autoencoder = Model(Input_img, decoded)
  #autoencoder.summary()
  autoencoder.compile(optimizer='adam', loss='mse')

  return autoencoder
def getNewModel():
  Input_img = Input(shape= patch_size + (3,))  
    
  #encoding architecture
  x1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(Input_img)
  x2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x1)
  x3 = MaxPool2D(padding='same')(x2)
  x4 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x3)
  x5 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x4)
  x6 = MaxPool2D(padding='same')(x5)
  encoded = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x6)
  #encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)
  # decoding architecture
  x7 = UpSampling2D()(encoded)
  x8 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x7)
  x9 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x8)
  x10 = Add()([x5, x9])
  x11 = UpSampling2D()(x10)
  x12 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x11)
  x13 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x12)
  x14 = Add()([x2, x13])
  # x3 = UpSampling2D((2, 2))(x3)
  # x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
  # x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
  decoded = Conv2D(3, (3, 3), padding='same',activation='relu', kernel_regularizer=regularizers.l1(10e-10))(x14)
  autoencoder = Model(Input_img, decoded)
  autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  autoencoder.summary()



Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 100, 100, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 100, 100, 64) 36928       conv2d_7[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 50, 50, 64)   0           conv2d_8[0][0]                   
_______________________________________________________________________________________

In [13]:
#@title
# from each of the generators create a pair of interleaved datasets
# tensorflow can automatically multiprocess interleaved datasets
# so that while batches can be loaded and processed ahead of time
interleaved_train = tf.data.Dataset.range(2).interleave(
                    get_train_dataset,
                    num_parallel_calls=tf.data.experimental.AUTOTUNE
                )
interleaved_val = tf.data.Dataset.range(2).interleave(
                    get_val_dataset,
                    num_parallel_calls=tf.data.experimental.AUTOTUNE
                )

In [14]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=3, restore_best_weights=True),
tf.keras.callbacks.ModelCheckpoint("data/model_checkpoints/weights.{epoch:02d}-{val_accuracy:.2f}.hdf5", monitor='val_accuracy', mode='max', save_best_only=True, save_weights_only=True)]

autoencoder = getNewModel()

a_e = autoencoder.fit(interleaved_train,
                   steps_per_epoch = train_steps,
                   epochs = 5,
                    verbose=1,
                    validation_data=interleaved_val,
                    validation_steps=val_steps,
                   callbacks=callbacks)

autoencoder.save("data/saved_models/autoencoder_large.h5")

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 100, 100, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 100, 100, 64) 36928       conv2d_17[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 50, 50, 64)   0           conv2d_18[0][0]                  
_______________________________________________________________________________________

AttributeError: 'NoneType' object has no attribute 'fit'

In [ ]:
from tqdm import tqdm

def reconstruct_image(patches, ranges, weights):
  img = np.zeros(weights.shape)
  for i, (y0, y1, x0, x1) in enumerate(ranges):
    img[y0:y1,x0:x1] += patches[i]
  
  return np.array((img / weights) * 255, dtype=np.uint8)

def sharpen_image(model, img):
  patches = []
  weights = np.zeros(img.shape)
  ranges = []
  for y in range(patch_size[1], img.shape[0] + 1 + (y_stride - (img.shape[0] % y_stride)), y_stride):
    if y > img.shape[0]:
      y = img.shape[0]

    for x in range(patch_size[0], img.shape[1] + 1 + (x_stride - (img.shape[1] % x_stride)), x_stride):
      if x > img.shape[1]:
        x = img.shape[1]

      y0 = y - patch_size[1]
      x0 = x - patch_size[0]
      patches.append(img[y0 : y, x0 : x])
      weights[y0 : y, x0 : x] += 1
      ranges.append([y0, y, x0, x])

  patches = np.array(patches)
  patch_batch_count = batch_size * max_patches
  preds = []
  for i in tqdm(list(range(0, patches.shape[0], patch_batch_count))):
    preds.append(np.array(model.predict(patches[i : i + patch_batch_count])))
  
  preds = np.concatenate(preds)

  return reconstruct_image(preds, ranges, weights)

def pixalate_image(image, scale_percent = 40):
      width = int(image.shape[1] * scale_percent / 100)
      height = int(image.shape[0] * scale_percent / 100)
      dim = (width, height)

      small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
      
      # scale back to original size
      width = image.shape[1]
      height = image.shape[0]
      dim = (width, height)

      low_res_image = cv2.resize(small_image, dim, interpolation = cv2.INTER_AREA)

      return low_res_image
  


In [ ]:
"""from tensorflow.keras import models 
autoencoder = models.load_model("/content/drive/My Drive/frame_interpolation/test_encoder2.h5")"""

In [ ]:
import cv2
img = keras.preprocessing.image.load_img("/content/drive/My Drive/frame_interpolation/test3.png")
img = keras.preprocessing.image.img_to_array(img) / 255
dim = (img.shape[1] * 2, img.shape[0] * 2)
print(img.shape)
img = pixalate_image(img, 25)#cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
img = np.array(img)
imageio.imwrite("/content/drive/My Drive/frame_interpolation/test3Pixelated.png", np.array(img * 255, dtype=np.uint8))
print(img.shape)
imshow(img)

In [ ]:
sharp_img = sharpen_image(autoencoder, img)
imshow(sharp_img)

In [ ]:
imageio.imsave("/content/drive/My Drive/frame_interpolation/test3out.png", sharp_img)